In [ ]:
import io

import pandas as pd
import numpy as np

from scipy.spatial.distance import cdist 

import utils

## Notes

- Channel 1:  b9 density of spots
- Channel 2:  b1 density of spots
- Channel 3:  b4 density of spots
- Channel 4:  b9-b1 colocalization density of spots
- Channel 5:  b1-b4 colocalization density of spots
- Channel 6:  b9-b4 colocalization density of spots
- Channel 7:  b9-b1-b4 colocalization density of spots
 
 
 
If something is in channel 7, it should show up only in channel 7, and not show up in any other channel (and right now, it shows up in all other channels, because if a spot has all three things, then by definition it also has each combination of 2, and each single channel).

If something is in channel 6, it should show up only in 6, and be subtracted from showing up in channels 1 and 3.

If something is in channel 5, it should show up only in 5, and be subtracted from spots in channels 2 and 3.

If something is in channel 4, it should show up only in 4, and be subtracted from spots in channels 1 and 2.

Then spots left over should show up in either channel 1,2, or 3.

## CSV File
Type the path to the csv file you want to analyze between the quotes after filename =


In [ ]:
csvfile = '/path/to/file.csv'
df = pd.read_csv(csvfile)
df['display'] = len(df)*[1]
df.tail()

In [ ]:
# convert x, y to integers for pixel coordinates
df['xi'] = df['X'].astype(np.int32)
df['yi'] = df['Y'].astype(np.int32)

# create the output image
### Doing this to get the max x and y to create image dimensions
dx = np.ceil(df['X'].max()) #+ 25
dy = np.ceil(df['Y'].max()) #+ 25
dx = dx.astype(np.int32)
dy = dy.astype(np.int32)



In [ ]:
#Group different channels
gdf = df.groupby('Ch')

#create a dictionary of channels
dfdict = {i: gdf.get_group(i).reset_index() for i in df.Ch.unique()}

array_dict = {ch : utils.coord_array(dfdict, ch) for ch in dfdict.keys()}

In [ ]:
### do channel 7
min_distances = utils.min_distances
d71_near, _ = min_distances(array_dict[7], array_dict[1])
d72_near, _ = min_distances(array_dict[7], array_dict[2])
d73_near, _ = min_distances(array_dict[7], array_dict[3])

dfdict[1].loc[d71_near, 'display'] = 0
dfdict[2].loc[d72_near, 'display'] = 0
dfdict[3].loc[d73_near, 'display'] = 0


### do channel 6- combo of 1 and 3
d61_near, _ = min_distances(array_dict[6], array_dict[1])
d63_near, _ = min_distances(array_dict[6], array_dict[3])

dfdict[1].loc[d61_near, 'display'] = 0
dfdict[3].loc[d63_near, 'display'] = 0

### do channel 5 - combo of 2 and 3
d52_near, _ = min_distances(array_dict[5], array_dict[2])
d53_near, _ = min_distances(array_dict[5], array_dict[3])

dfdict[2].loc[d52_near, 'display'] = 0
dfdict[3].loc[d53_near, 'display'] = 0

### do channel 4 - combo of 1, and 2
d41_near, _ = min_distances(array_dict[4], array_dict[1])
d42_near, _ = min_distances(array_dict[4], array_dict[2])

dfdict[1].loc[d41_near, 'display'] = 0
dfdict[2].loc[d42_near, 'display'] = 0

'''
For channels 4,5,6, and 7 - go from 4 to 7 and find the closest
from each spot and turn it off

Cross of the above - if its in channel seven, remove it from the others
'''
### do channel 7 - combo of 6, and 5, and 4
d74_near, d74_dist = min_distances(array_dict[7], array_dict[4])
d75_near, d75_dist = min_distances(array_dict[7], array_dict[5])
d76_near, d76_dist = min_distances(array_dict[7], array_dict[6])

dfdict[4].loc[d74_near, 'display'] = 0
dfdict[5].loc[d75_near, 'display'] = 0
dfdict[6].loc[d76_near, 'display'] = 0


In [ ]:
set_channel_pixels = utils.set_channel_pixels
g = np.zeros((dy, dx, 7), dtype=np.float32)
set_channel_pixels(dfdict[7], 7, g)
set_channel_pixels(dfdict[6], 6, g)
set_channel_pixels(dfdict[5], 5, g)
set_channel_pixels(dfdict[4], 4, g)
set_channel_pixels(dfdict[1], 1, g, display=1)
set_channel_pixels(dfdict[2], 2, g, display=1)
set_channel_pixels(dfdict[3], 3, g, display=1)

g = (g - g.min())/(g.max() - g.min())

### If you know the image size, you can enter it below before saving
Replace `dx` and `dy` below with the actual image width and height

In [ ]:
import tifffile
image_size_x = 2048
image_size_y = 2044

g_resized = np.zeros((image_size_y, image_size_x, g.shape[-1]), dtype=np.float32)
g_resized[0:dy, 0:dx, :] = g
gw = np.moveaxis(g_resized, -1, 0).astype(np.float32)
tifffile.imwrite("//n/core/micro/rek/zaf/20201222_3PO_F1wt-Hobbit647-Blinc555-b4atto488/PROCESSED/measurements/CJW_spots/Tail2_3rdRow-CJWspots.tif",
                 gw, imagej=True)